In [1]:
import pandas as pd
import numpy as np
import os

# Zip Code Level Data

We first calculate the number of farm operations. This data is from two seperate files, so the data are added to each other with missing values set to zero to produce the final numbers

In [2]:
df_farms_1 = pd.read_csv("Farms 1.csv")

In [3]:
df_farms_1["Zip Code"] = df_farms_1["Zip Code"].astype(str).str.zfill(5)
df_farms_1['Value'] = pd.to_numeric(df_farms_1['Value'].str.replace(',', ''), errors='coerce')
df_farms_1.rename(columns={'Value': 'Num Farms'}, inplace=True)
grouped_df = df_farms_1.groupby('Zip Code')['Num Farms'].sum().reset_index()
grouped_df.set_index('Zip Code', inplace=True)

In [4]:
df_farms_2 = pd.read_csv("Farms 2.csv")
df_farms_2["Zip Code"] = df_farms_2["Zip Code"].astype(str).str.zfill(5)
df_farms_2['Value'] = pd.to_numeric(df_farms_2['Value'].str.replace(',', ''), errors='coerce')
df_farms_2.rename(columns={'Value': 'Num Farms'}, inplace=True)

In [5]:
grouped_df_2 = df_farms_2.groupby('Zip Code')['Num Farms'].sum().reset_index()
grouped_df_2.set_index('Zip Code', inplace=True)

In [6]:
df_farms = grouped_df.add(grouped_df_2, fill_value=0)

The zip code 99999 represents areas which are not included in any zip code (military bases, federal lands, etc.). This line will be removed prior to statistical analysis but is computationally convenient to leave in the data until the end due to its presence in other data files.

Next, use data from the CBP to determine the number of employer establishments in each zip code. This does not include farms, so it is added to the farm data to produce a total employer establishment estimate.

In [7]:
cbp_df = pd.read_csv("zbp20totals.csv", encoding='latin1')
cbp_df["zip"] = cbp_df["zip"].astype(str).str.zfill(5)
cbp_df.set_index("zip", inplace=True)

In [8]:
cbp_df.drop("99999").describe()

,emp,qp1,ap,est
count,35052.000000,3.505200e+04,3.505200e+04,35052.000000
mean,3638.925140,5.409963e+04,2.047119e+05,227.137111
std,8085.098199,2.014584e+05,6.944561e+05,406.851597
min,0.000000,0.000000e+00,1.100000e+01,3.000000
25%,80.000000,6.810000e+02,2.958000e+03,11.000000
50%,423.000000,4.163500e+03,1.716400e+04,41.000000
75%,3367.000000,3.543050e+04,1.424528e+05,255.000000
max,177226.000000,9.898336e+06,2.627020e+07,6893.000000


In [9]:
master_df = pd.concat([df_farms, cbp_df[["est", "emp"]]], axis=1)
master_df = master_df.sort_index()
master_df.rename(columns={'est': 'Num Estabs', "emp":"CBP Emp"}, inplace=True)
master_df["Total Emp Bus"] = master_df["Num Farms"].add(master_df["Num Estabs"], fill_value=0)

Next, we calculate the percentage of the population that is held by various racial and ethnic groups for each zip code.

In [10]:
race_df = pd.read_csv("Race By ZIP.csv", header = 1)
race_df["Zip"] = race_df["Geography"].str.slice(-5)
race_df.set_index("Zip", inplace=True)

In [11]:
race_df["Min Share"] = 1 - (race_df[" !!Total:!!Not Hispanic or Latino:!!Population of one race:!!White alone"] / race_df[" !!Total:"])
race_df["Black Share"] = race_df[" !!Total:!!Not Hispanic or Latino:!!Population of one race:!!Black or African American alone"] / race_df[" !!Total:"]
race_df["Hisp Share"] = race_df[" !!Total:!!Hispanic or Latino"] / race_df[" !!Total:"]
race_df["Asian Share"] = race_df[" !!Total:!!Not Hispanic or Latino:!!Population of one race:!!Asian alone"] / race_df[" !!Total:"]
race_df["Native Share"] = race_df[" !!Total:!!Not Hispanic or Latino:!!Population of one race:!!White alone"] / race_df[" !!Total:"]
race_df["White Share"] =  race_df[" !!Total:!!Not Hispanic or Latino:!!Population of one race:!!American Indian and Alaska Native alone"] / race_df[" !!Total:"]

C:\Users\hthog\AppData\Local\Temp\ipykernel_14324\1813367382.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  race_df["Min Share"] = 1 - (race_df[" !!Total:!!Not Hispanic or Latino:!!Population of one race:!!White alone"] / race_df[" !!Total:"])
C:\Users\hthog\AppData\Local\Temp\ipykernel_14324\1813367382.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  race_df["Black Share"] = race_df[" !!Total:!!Not Hispanic or Latino:!!Population of one race:!!Black or African American alone"] / race_df[" !!Total:"]
C:\Users\hthog\AppDa

In [12]:
master_df = pd.concat([master_df, race_df[["Min Share", "Black Share", "Hisp Share", 
                                           "Asian Share", "Native Share", "White Share", 
                                           " !!Total:"]]], axis=1)
master_df.rename(columns={' !!Total:': 'Total Pop'}, inplace=True)
master_df = master_df.sort_index()

In [13]:
master_df.drop("99999", inplace=True)
master_df.tail()

,Num Farms,Num Estabs,CBP Emp,Total Emp Bus,Min Share,Black Share,Hisp Share,Asian Share,Native Share,White Share,Total Pop
99926,1.0,11.0,51.0,12.0,0.903329,0.002561,0.029449,0.000000,0.096671,0.743278,1562.0
99927,NaN,NaN,NaN,NaN,0.142857,0.000000,0.040816,0.000000,0.857143,0.000000,49.0
99928,1.0,4.0,15.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99929,2.0,93.0,474.0,95.0,0.364598,0.000000,0.036075,0.012025,0.635402,0.173641,2079.0
99950,3.0,12.0,43.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
master_df.describe()

,Num Farms,Num Estabs,CBP Emp,Total Emp Bus,Min Share,Black Share,Hisp Share,Asian Share,Native Share,White Share,Total Pop
count,32109.000000,35052.000000,35052.000000,37322.000000,33631.000000,33631.000000,33631.000000,33631.000000,33631.000000,33631.000000,33774.000000
mean,61.825968,227.137111,3638.925140,266.512513,0.263090,0.073105,0.104150,0.023165,0.736910,0.017748,9910.777107
std,88.443966,406.851597,8085.098199,414.475046,0.248661,0.148541,0.163993,0.057624,0.248661,0.092316,14918.152573
min,1.000000,3.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9.000000,11.000000,80.000000,24.000000,0.076952,0.002643,0.018634,0.001500,0.625929,0.000821,650.000000
50%,31.000000,41.000000,423.000000,90.000000,0.162887,0.010017,0.040991,0.005085,0.837113,0.002203,2649.500000
75%,78.000000,255.000000,3367.000000,326.000000,0.374071,0.063405,0.107638,0.017196,0.923048,0.005195,13359.000000
max,1634.000000,6893.000000,177226.000000,6897.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,135256.000000


# County Level Data

In [15]:
county_df = pd.read_csv("ZIP-COUNTY-FIPS_2017-06.csv")
county_df["ZIP"] = county_df["ZIP"].astype(str).str.zfill(5)
county_df["STCOUNTYFP"] = county_df["STCOUNTYFP"].astype(str).str.zfill(5)

In [16]:
county_df_grouped = county_df.groupby('ZIP')['STCOUNTYFP'].agg(list).reset_index()
county_df_grouped.set_index('ZIP', inplace=True)

In [17]:
master_df = pd.concat([master_df, county_df_grouped], axis=1)

In [18]:
emp_df = pd.read_csv("Employer Race by County.csv", header = 1)
emp_df = emp_df[["Geographic identifier code", 
                 "Geographic Area Name", 
                 "Sex code", 
                 "Meaning of Sex code", 
                 "Ethnicity code",
                 "Meaning of Ethnicity code", 
                 "Race code",
                 "Meaning of Race code",
                 "Veteran code",
                 "Meaning of Veteran code",
                 "Year",
                 "Number of employer firms"]]
emp_df["FIPS"] = emp_df["Geographic identifier code"].str.slice(-5)

In oder to filter rows to just those needed, we filter the values of the other characteristic columns to those only identified as totals.

In [19]:
emp_df_filtered = emp_df[emp_df["Sex code"] == 1]
emp_df_filtered = emp_df_filtered[emp_df_filtered["Ethnicity code"] == 1]
emp_df_filtered = emp_df_filtered[emp_df_filtered["Veteran code"] == 1]
emp_df_filtered["Number of employer firms"].replace("S", np.nan, inplace=True)

In [20]:
index = sorted(list(set(emp_df_filtered["FIPS"])))
columns = list(set(emp_df_filtered["Meaning of Race code"]))
owner_df = pd.DataFrame(index = index, columns = columns).fillna(0)

In [21]:
for column in owner_df.columns:
    sub_df = emp_df_filtered[emp_df_filtered["Meaning of Race code"] == column]
    sub_df.set_index("FIPS", inplace=True)
    owner_df[column] = sub_df["Number of employer firms"].astype(np.double)

Becaude Hispanic is accounted for in another column, we calculate their share of employer businesses seperately.

In [22]:
emp_df_2 = emp_df[emp_df["Sex code"] == 1]
emp_df_2 = emp_df_2[emp_df_2["Veteran code"] == 1]
emp_df_2 = emp_df_2[emp_df_2["Race code"] == 0]
emp_df_2["Number of employer firms"].replace("S", np.nan, inplace=True)

In [23]:
owner_df[["Hispanic", "Non-Hispanic"]] = 0

In [24]:
for category in ["Hispanic", "Non-Hispanic"]:
    sub_df = emp_df_2[emp_df_2["Meaning of Ethnicity code"] == category]
    sub_df.set_index("FIPS", inplace=True)
    owner_df[category] = sub_df["Number of employer firms"].astype(np.double)

In [25]:
owner_df["Emp Min Share"] = owner_df["Minority"] / owner_df["Total"]
owner_df["Emp Black Share"] = owner_df["Black or African American"] / owner_df["Total"]
owner_df["Emp White Share"] = owner_df["White"] / owner_df["Total"]
owner_df["Emp Asian Share"] = owner_df["Asian"] / owner_df["Total"]
owner_df["Emp Hisp Share"] = owner_df["Hispanic"] / owner_df["Total"]
owner_df["Emp Non-Hisp Share"] = owner_df["Non-Hispanic"] / owner_df["Total"]
owner_df.describe()

,Minority,Black or African American,Asian,Native Hawaiian and Other Pacific Islander,White,Total,Equally minority/nonminority,American Indian and Alaska Native,Nonminority,Hispanic,Non-Hispanic,Emp Min Share,Emp Black Share,Emp White Share,Emp Asian Share,Emp Hisp Share,Emp Non-Hisp Share
count,1041.000000,370.000000,666.000000,37.000000,2667.000000,2972.000000,313.000000,173.000000,2652.000000,477.000000,2686.000000,1031.000000,367.000000,2639.000000,664.000000,475.000000,2657.000000
mean,939.705091,285.440541,785.462462,95.324324,1855.201350,2164.483513,220.725240,75.300578,1712.736425,609.817610,1969.518243,0.115679,0.029252,0.798482,0.066516,0.056190,0.823044
std,3604.659095,511.731815,2782.302697,161.231210,5714.036688,7324.429619,445.289248,107.058188,4952.981741,2099.655255,6396.526661,0.087411,0.031003,0.071444,0.052491,0.073591,0.068547
min,20.000000,20.000000,21.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,0.013952,0.002319,0.357963,0.008291,0.003944,0.183206
25%,63.000000,40.250000,69.250000,31.000000,210.000000,221.000000,42.000000,31.000000,202.000000,56.000000,208.000000,0.058591,0.011579,0.761289,0.035497,0.018301,0.798561
50%,158.000000,106.000000,168.500000,46.000000,487.000000,520.500000,94.000000,44.000000,475.000000,145.000000,496.500000,0.092402,0.020918,0.807671,0.053566,0.033228,0.834123
75%,510.000000,289.750000,507.750000,82.000000,1301.500000,1387.500000,219.000000,79.000000,1268.250000,390.000000,1344.750000,0.141300,0.033303,0.845813,0.076210,0.065245,0.862944
max,81550.000000,4408.000000,55036.000000,884.000000,156111.000000,228558.000000,5282.000000,961.000000,130465.000000,33891.000000,192444.000000,0.679245,0.304630,1.000000,0.534213,0.602947,1.000000


In [26]:
missing_values_count = owner_df.isnull().sum()
missing_df = pd.DataFrame(missing_values_count)
missing_df["% missing"] = missing_df[0] / 3140
missing_df["# available"] = 3140 - missing_df[0]
missing_df

,0,% missing,# available
Minority,2099,0.668471,1041
Black or African American,2770,0.882166,370
Asian,2474,0.787898,666
Native Hawaiian and Other Pacific Islander,3103,0.988217,37
White,473,0.150637,2667
Total,168,0.053503,2972
Equally minority/nonminority,2827,0.900318,313
American Indian and Alaska Native,2967,0.944904,173
Nonminority,488,0.155414,2652
Hispanic,2663,0.848089,477


In [27]:
def calc_weighted_avg(fips_list, col_name):
    if not isinstance(fips_list, list):
        return None

    total_sum = 0
    weighted_sum = 0

    for fips in fips_list:
        if fips in owner_df.index:
            weighted_sum += owner_df.loc[fips, col_name] * owner_df.loc[fips, 'Total']
            total_sum += owner_df.loc[fips, 'Total']

    return weighted_sum / total_sum if total_sum != 0 else None

In [28]:
demographic_cols = ['Emp Min Share', 
                    'Emp Black Share',
                    'Emp White Share',
                    'Emp Asian Share',
                    'Emp Hisp Share',
                    'Emp Non-Hisp Share']

for col in demographic_cols:
    new_col_name = 'W_Avg ' + col
    master_df[new_col_name] = master_df['STCOUNTYFP'].apply(calc_weighted_avg, col_name=col)

In [29]:
master_df

,Num Farms,Num Estabs,CBP Emp,Total Emp Bus,Min Share,Black Share,Hisp Share,Asian Share,Native Share,White Share,Total Pop,STCOUNTYFP,W_Avg Emp Min Share,W_Avg Emp Black Share,W_Avg Emp White Share,W_Avg Emp Asian Share,W_Avg Emp Hisp Share,W_Avg Emp Non-Hisp Share
00501,NaN,5.0,49.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[36103],0.149040,NaN,0.859492,0.083086,0.056364,0.892998
00601,NaN,NaN,NaN,NaN,0.997854,0.000058,0.996694,0.000000,0.002146,0.00000,17242.0,"[72001, 72113]",NaN,NaN,NaN,NaN,NaN,NaN
00602,NaN,NaN,NaN,NaN,0.994327,0.000266,0.992809,0.000266,0.005673,0.00008,37548.0,"[72003, 72005]",NaN,NaN,NaN,NaN,NaN,NaN
00603,NaN,NaN,NaN,NaN,0.988274,0.001606,0.983776,0.000984,0.011726,0.00010,49804.0,"[72005, 72071]",NaN,NaN,NaN,NaN,NaN,NaN
00606,NaN,NaN,NaN,NaN,0.994809,0.000200,0.993013,0.000399,0.005191,0.00000,5009.0,"[72093, 72153]",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99252,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[53063],0.061404,NaN,0.852339,0.029630,0.020760,0.864522
99256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[53063],0.061404,NaN,0.852339,0.029630,0.020760,0.864522
99529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[02020],0.135079,0.015303,0.751177,0.072248,0.027810,0.826074
99599,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[02020],0.135079,0.015303,0.751177,0.072248,0.027810,0.826074


In [30]:
master_df.to_csv("ZIPdf.csv")

In [31]:
def weighted_average(df, values_col, weights_col):
    # Element-wise multiplication, ignoring NaNs
    weighted_values = df[values_col] * df[weights_col]
    
    # Sum of weighted values and weights, ignoring NaNs
    sum_weighted_values = weighted_values.sum(skipna=True)
    sum_weights = df[weights_col].sum(skipna=True)
    
    # Calculate weighted average, returning NaN if not computable
    if sum_weights == 0:
        return np.nan
    else:
        return sum_weighted_values / sum_weights

weighted_average(master_df, "Min Share", "Total Pop")

0.42723896750764817

<p><b>Release Date:</b> 2020-05-19</p>

<p>The Census Bureau has reviewed this data product for unauthorized disclosure of confidential information and has approved the disclosure avoidance practices applied (Approval ID: CBDRB-FY20-008).</p>

<p><b>Release Schedule:</b><br />
Data in this file come from estimates of business ownership by sex, ethnicity, race, and veteran status from the 2018 Annual Business Survey (ABS) collection. Data are also obtained from administrative records, the 2017 Economic Census and other economic surveys.</p>

<p><i>Note:</i> The collection year is the year in which the data are collected. A reference year is the year that is referenced in the questions on the survey and in which the statistics are tabulated. For example, the 2018 ABS collection year produces statistics for the 2017 reference year. The "Year" column in the table is the reference year. The ABS has a larger sample size during the benchmark year of 2017. Due to the larger size, more detailed data are shown for reference year 2017.</p>

<p>For more information about ABS planned data product releases, see <a href="https://www.census.gov/programs-surveys/abs/newsroom/updates/tentative-schedule.html">Tentative ABS Schedule</a>.</p>

<p><b>Key Table Information:</b><br />
Includes U.S. firms with paid employees, operating during the reference year with receipts of $1,000 or more, which are classified in the North American Industry Classification System (NAICS), Sectors 11 through 99, except for NAICS 111, 112, 482, 491, 521, 525, 813, 814, and 92 which are not covered. Employer firms with more than one domestic establishment are counted in each geographic area and industry in which they operate, but only once in the U.S. and state totals for all sectors. Employment reflects the number of paid employees during the pay period in the reference year that included March 12.</p>

<p><b>Data Items and Other Identifying Records:</b><br />
Data include estimates on:
<ul><li>Number of employer firms (firms with paid employees)</li>
    <li>Sales and receipts of employer firms (reported in $1,000s of dollars)</li>
    <li>Number of employees (during the March 12 pay period)</li>
    <li>Annual payroll (reported in $1,000s of dollars)</li>
</ul></p>

<p>These data are aggregated by the following demographic classifications of firm for:
<ul><li>All firms
    <ul><li>Classifiable (firms classifiable by sex, ethnicity, race, and veteran status)
    <ul>
	<li>Sex
        <ul><li>Female</li>
            <li>Male</li>
            <li>Equally male/female</li>
        </ul></li>
    <li>Ethnicity
	    <ul><li>Hispanic</li>
            <li>Equally Hispanic/non-Hispanic</li>
            <li>Non-Hispanic</li>
        </ul></li>
    <li>Race
	    <ul><li>White</li>
            <li>Black or African American</li>
            <li>American Indian and Alaska Native</li>
            <li>Asian</li>
            <li>Native Hawaiian and Other Pacific Islander</li>
            <li>Minority (Firms classified as any race and ethnicity combination other than non-Hispanic and White)</li>
            <li>Equally minority/nonminority</li>
            <li>Nonminority (Firms classified as non-Hispanic and White)</li>
        </ul></li>
    <li>Veteran Status (defined as having served in any branch of the U.S. Armed Forces)
	    <ul><li>Veteran</li>
            <li>Equally veteran/nonveteran</li>
            <li>Nonveteran</li>
        </ul></li>
    </ul>
	</li>
        <li>Unclassifiable (firms not classifiable by sex, ethnicity, race, and veteran status)</li>
    </ul></li>
</ul></p>

<p>Moreover, the 2017 reference year statistics include detailed race and ethnicity data tabulated for:
<ul><li>Hispanic subgroups
    <ul><li>Mexican, Mexican American, Chicano</li>
        <li>Puerto Rican</li>
        <li>Cuban</li>
        <li>Other Hispanic, Latino, or Spanish</li>
    </ul></li>
    <li>Asian subgroups
    <ul><li>Asian Indian</li>
        <li>Chinese</li>
        <li>Filipino</li>
        <li>Japanese</li>
        <li>Korean</li>
        <li>Vietnamese</li>
        <li>Other Asian</li>
    </ul></li>
    <li>Native Hawaiian and Other Pacific Islander subgroups
	<ul><li>Native Hawaiian</li>
	<li>Guamanian or Chamorro</li>
        <li>Samoan</li>
        <li>Other Pacific Islander</li>
    </ul></li>
</ul></p>

<p><i>Data Notes:</i>
<ol>
    <li>Business ownership is defined as having 51 percent or more of the stock or equity in the business. Data are provided for businesses owned equally (50% / 50%) by men and women, by Hispanics and non-Hispanics, by minorities and nonminorities, and by veterans and nonveterans. Firms not classifiable by sex, ethnicity, race, and veteran status are counted and tabulated separately.</li>
    <li>The detail may not add to the total or subgroup total because a Hispanic or Latino firm may be of any race, and because a firm could be tabulated in more than one racial group. For example, if a firm responded as both Chinese and Black majority owned, the firm would be included in the detailed Asian and Black estimates but would only be counted once toward the higher level all firms' estimates.</li>
    <li>References such as "Mexican-owned," "Puerto Rican-owned," "Cuban-owned" or "other Hispanic- or Latino-owned" businesses refer only to businesses operating in the 50 states and the District of Columbia that self-identified 51 percent or more of their ownership in 2017 to be by individuals of Mexican, Puerto Rican, Cuban or other Hispanic or Latino origin. The ABS does not distinguish between U.S. residents and nonresidents. Companies owned by foreign governments or owned by other companies, foreign or domestic, are included in the category "Unclassifiable."</li>
</ol></p>

<p><b>Industry and Geography Coverage:</b><br />

The data are shown for the total for all sectors (00) and the 2-, 3-, 4-, 5-, and 6-digit NAICS code levels for:

<ul><li>United States</li>
    <li><a href="https://www.census.gov/programs-surveys/economic-census/geographies/levels/2017-levels.html#par_textimage_38">States and the District of Columbia</a></li>
</ul></p>

<p>In addition, the total of all sectors (00) and the 2-digit NAICS code levels are shown for:

<ul><li>Metro Areas, including:
    <ul>
        <li><a href="https://www.census.gov/programs-surveys/economic-census/geographies/levels/2017-levels.html#par_textimage_6">Metropolitan Statistical Areas</a></li>
        <li><a href="https://www.census.gov/programs-surveys/economic-census/geographies/levels/2017-levels.html#par_textimage_7">Micropolitan Statistical Areas</a></li>
        <li><a href="https://www.census.gov/programs-surveys/economic-census/geographies/levels/2017-levels.html#par_textimage_8">Metropolitan Divisions</a></li>
        <li><a href="https://www.census.gov/programs-surveys/economic-census/geographies/levels/2017-levels.html#par_textimage_9">Combined Statistical Areas</a></li>
    </ul></li>
    <li><a href="https://www.census.gov/programs-surveys/economic-census/geographies/levels/2017-levels.html#par_textimage">Counties</a></li>
    <li><a href="https://www.census.gov/programs-surveys/economic-census/geographies/levels/2017-levels.html#par_textimage_19">Economic Places</a></li>
</ul></p>

<p>For more information about NAICS, see <a href="https://www.census.gov/programs-surveys/economic-census/guidance/understanding-naics.html">NAICS Codes & Understanding Industry Classification Systems</a>. For information about geographies used by economic programs at the Census Bureau, see <a href="https://www.census.gov/programs-surveys/economic-census/geographies.html">Economic Census: Economic Geographies.</a></p>

<p><b>Footnotes:</b><br />
Footnote 660 - Agriculture, forestry, fishing and hunting (Sector 11): Crop and Animal Production (NAICS 111 and 112) are out of scope.<br />
Footnote 661 - Transportation and warehousing (Sector 48-49): Rail Transportation (NAICS 482) and the Postal Service (NAICS 491) are out of scope.<br />
Footnote 662 - Finance and insurance (Sector 52): Monetary Authorities-Central Banks (NAICS 521) and Funds, Trusts, and Other Financial Vehicles (NAICS 525) are out of scope.<br />
Footnote 663 - Other services, except public administration (Sector 81): Religious, Grantmaking, Civic, Professional, and Similar Organizations (NAICS 813) and Private Households (NAICS 814) are out of scope.</p>

<p><b>FTP Download:</b><br />
Download the entire table at: <a href="https://www2.census.gov/programs-surveys/abs/data/2017/AB1700CSA01.zip">https://www2.census.gov/programs-surveys/abs/data/2017/AB1700CSA01.zip</a>.</p>

<p><b>API Information:</b><br />
Annual Business Survey data are housed in the Census Bureau API. For more information, see <a href="https://api.census.gov/data/2017/abscs.html ">https://api.census.gov/data/2017/abscs.html</a>.</p>

<p><b>Methodology:</b><br />
To maintain confidentiality, the Census Bureau suppresses data to protect the identity of any business or individual. The census results in this file contain sampling and/or nonsampling error. Data users who create their own estimates using data from this file should cite the Census Bureau as the source of the original data only. For information on confidentiality protection, sampling error, nonsampling error, and definitions, see <a href="https://www.census.gov/programs-surveys/abs/technical-documentation/methodology.html">Survey Methodology</a>.</p>

<p><b>Symbols:</b><br />
 <b>S</b> - Estimate does not meet publication standards because of high sampling variability, poor response quality, or other concerns about the estimate quality. Unpublished estimates derived from this table by subtraction are subject to these same limitations and should not be attributed to the U.S. Census Bureau. For a description of publication standards and the total quantity response rate, see link to program methodology <a href="https://www.census.gov/programs-surveys/abs/technical-documentation/methodology.html">page</a>.<br />
 <b>N</b> - Not available or not comparable<br />
 <b>X</b> - Not applicable<br />
For a complete list of all economic programs symbols, see the <a href="https://www.census.gov/programs-surveys/economic-census/technical-documentation/data-dictionary.html">Symbols Glossary</a>.</p>

<p><b>Source:</b><br />
U.S. Census Bureau, Annual Business Survey (ABS)<br />
For more information about the survey, please visit <a href="https://www.census.gov/programs-surveys/abs.html">https://www.census.gov/programs-surveys/abs.html</a>.</p>

<p><b>Contact Information:</b><br />
To contact the Annual Business Survey staff:
<ul><li>Email general, nonsecure, and unencrypted messages to <a href="mailto:adep.annual.business.survey@census.gov">adep.annual.business.survey@census.gov</a>.</li>
    <li>Call 301.763.3316 between 7 a.m. and 5 p.m. (EST), Monday through Friday.</li>
</ul>
</p>
